In [ ]:
# 🚀 Auto-setup: installs deps + configures CFBD access
%run ./_auto_setup.py


In [ ]:
# 01_intro_to_data.ipynb
# 📘 Introduction to College Football Data Starter Pack

# ---
# 👋 Welcome!
# This notebook introduces you to the structure and content of the CFB Starter Pack.
# You'll learn how to load game data and link it to play and drive data.
# All data is provided as CSVs for easy access — no API setup required.

# 🧰 Requirements:
# - Python 3.8+
# - pandas, matplotlib (install via `pip install pandas matplotlib`)

import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
from pathlib import Path

# Import starter pack configuration system
_config_dir = Path().resolve() / "config"
if str(_config_dir.parent) not in sys.path:
    sys.path.insert(0, str(_config_dir.parent))
from config.data_config import get_starter_pack_config

# Get configuration
config = get_starter_pack_config()
DATA_DIR = str(config.data_dir)

In [ ]:
# 📂 Load games.csv — your entry point (with optional live fetch)

fetch_latest = False  # 👈 Flip to True to pull current-season data from CFBD

if fetch_latest:
    from starter_pack.utils.cfbd_loader import CFBDLoader, CFBDLoaderError

    loader = CFBDLoader()
    try:
        games = loader.fetch_games(season=config.current_year, season_type="regular")
        print(f"🌐 Loaded {len(games):,} games directly from CFBD for {config.current_year}.")
    except CFBDLoaderError as err:
        print(f"⚠️ Live fetch failed ({err}); falling back to local CSV...")
        fetch_latest = False

if not fetch_latest:
    games_path = os.path.join(DATA_DIR, "games.csv")
    games = pd.read_csv(games_path)
    print(f"📁 Loaded {len(games):,} total games from {games_path}")

games.head()

In [ ]:
# Set the display options for better readability
plt.style.use('fivethirtyeight')
plt.rcParams["figure.figsize"] = [20,10]

In [ ]:
# 🔍 Explore games by season
games['season'].value_counts().sort_index().plot(kind='bar', figsize=(12,4), title='Games per Season')
plt.ylabel('Game Count')
plt.xlabel('Season')
plt.grid(axis='y')
plt.show()

In [ ]:
# 🎯 Let's zoom in on a specific game
# Example: Recent National Championship (Michigan vs. Washington)
# Use the most recent season available in the data

current_year = config.current_year
# Find the most recent championship game with these teams
recent_seasons = sorted(games['season'].unique(), reverse=True)[:3]  # Check last 3 seasons

sample_game = None
for season in recent_seasons:
    potential_game = games[
        (games['season'] == season) &
        (games['season_type'] == 'postseason') &
        (games['home_team'] == "Michigan") &
        (games['away_team'] == "Washington")
    ]
    if len(potential_game) > 0:
        sample_game = potential_game.iloc[0]
        break

# Fallback: get any recent championship game if specific teams not found
if sample_game is None:
    recent_championship = games[
        (games['season_type'] == 'postseason') &
        (games['season'].isin(recent_seasons))
    ].iloc[0]
    sample_game = recent_championship

print(f"Selected Game ID: {sample_game['id']}")
sample_game


In [ ]:
# 📜 Load plays and drives for that game
# Note: Plays and drives are split into weekly CSVs

game_season = sample_game['season']
plays_path = config.get_plays_path(year=game_season, season_type="postseason")
drives_path = config.get_drives_path(year=game_season)

plays = pd.read_csv(plays_path)
drives = pd.read_csv(drives_path)

# Filter for this game
game_id = sample_game['id']
plays = plays.query(f"gameId == {game_id}")
drives = drives.query(f"gameId == {game_id}")

print(f"{len(plays):,} plays, {len(drives):,} drives")


In [ ]:
# 🧠 View a few plays
plays[['period', 'clock', 'offense', 'defense', 'down', 'distance', 'yardsToGoal', 'yardsGained', 'playText']].head(10)

In [ ]:
# 📊 Explore number of plays per drive

plays_per_drive = plays.groupby('driveNumber').size()
plays_per_drive.plot(kind='bar', title='Plays per Drive')
plt.xlabel("Drive ID")
plt.ylabel("Number of Plays")
plt.grid(axis='y')
plt.show()


In [ ]:
# Explore the structure of drives
drives.head()

In [ ]:
# Calculate average yards per drive for each team
avg_yards_per_drive = drives.groupby(['offense'])['yards'].mean().reset_index()
avg_yards_per_drive = avg_yards_per_drive.sort_values(by='yards', ascending=False)

avg_yards_per_drive

In [ ]:
# ✅ That’s it for now!

# You've just:
# - Loaded core game data
# - Linked a game to its drives and plays
# - Explored drive structure visually

# 🧪 Next steps:
# - Use `advanced_game_stats` to look at efficiency metrics
# - Join play-level data with outcomes (EPA, success rate, etc.)